# Scrape Wikipedia table

In [1]:
from bs4 import BeautifulSoup

In [2]:
import requests

In [3]:
import pandas as pd

In [4]:
wiki_url ='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [5]:
page = requests.get(wiki_url)
soup = BeautifulSoup(page.text, 'html.parser')

In [68]:
#print(soup)

In [6]:
my_table = soup.find_all('table',attrs={'class':'wikitable sortable'})

In [69]:
#print(my_table)

In [7]:
df = pd.read_html(str(my_table))

In [8]:
pd.set_option('display.max_columns',20)
pd.set_option('display.max_colwidth',30)
pd.set_option('display.max_rows',10)
print(df)

[    Postal Code           Borough                   Neighborhood
0           M1A      Not assigned                            NaN
1           M2A      Not assigned                            NaN
2           M3A        North York                      Parkwoods
3           M4A        North York               Victoria Village
4           M5A  Downtown Toronto      Regent Park, Harbourfront
..          ...               ...                            ...
175         M5Z      Not assigned                            NaN
176         M6Z      Not assigned                            NaN
177         M7Z      Not assigned                            NaN
178         M8Z         Etobicoke  Mimico NW, The Queensway W...
179         M9Z      Not assigned                            NaN

[180 rows x 3 columns]]


## Data cleaning

In [9]:
df1 = df[0]

#print ((df1['Borough'])!= 'Not assigned')
df2 = df1[df1.Borough != 'Not assigned']
df3 = df2.iloc[:,0:3]
print(df3)

    Postal Code           Borough                   Neighborhood
2           M3A        North York                      Parkwoods
3           M4A        North York               Victoria Village
4           M5A  Downtown Toronto      Regent Park, Harbourfront
5           M6A        North York  Lawrence Manor, Lawrence H...
6           M7A  Downtown Toronto  Queen's Park, Ontario Prov...
..          ...               ...                            ...
160         M8X         Etobicoke  The Kingsway, Montgomery R...
165         M4Y  Downtown Toronto           Church and Wellesley
168         M7Y      East Toronto  Business reply mail Proces...
169         M8Y         Etobicoke  Old Mill South, King's Mil...
178         M8Z         Etobicoke  Mimico NW, The Queensway W...

[103 rows x 3 columns]


In [10]:
df3.shape

(103, 3)

## Segmenting and Clustering Neighborhoods 

In [11]:
df4 = pd.DataFrame(df3)
df4

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence H..."
6,M7A,Downtown Toronto,"Queen's Park, Ontario Prov..."
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery R..."
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,Business reply mail Proces...
169,M8Y,Etobicoke,"Old Mill South, King's Mil..."


In [12]:
import numpy as np 

import json 

from pandas.io.json import json_normalize 

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

#import folium 

print('Libraries imported.')

Libraries imported.


In [13]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_92b1a7b939944c14add0427ffeb79e45 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='XLCjiEy4ETm8Zr51gSlMkGMf_a4GkNY_N_2cLyEIsh1s',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_92b1a7b939944c14add0427ffeb79e45.get_object(Bucket='segmentingandclusteringneighborho-donotdelete-pr-uayaijcsjl3hha',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

# If you are reading an Excel file into a pandas DataFrame, replace `read_csv` by `read_excel` in the next statement.
df_data_0 = pd.read_csv(body)
df_data_0.head()


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [14]:
df_both = pd.merge(df4,df_data_0, on='Postal Code')
df_both

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence H...",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Prov...",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery R...",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,Business reply mail Proces...,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mil...",43.636258,-79.498509


In [15]:
!pip install geopy

In [22]:
import geopy
print('geopy imported')

geopy imported


In [17]:

from geopy.geocoders import Nominatim
address = 'Toronto'

geolocator = Nominatim(user_agent = "Toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto are 43.6534817, -79.3839347.


In [30]:
!pip install folium
import folium 
print('folim imported.')

     |████████████████████████████████| 102kB 7.3MB/s ta 0:00:011
folim imported.


In [34]:

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)


for lat, lng, borough, neighborhood in zip(df_both['Latitude'], df_both['Longitude'], df_both['Borough'], df_both['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto